<a href="https://colab.research.google.com/github/vincent0965/Colab_note/blob/master/sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

dataset = tf.keras.utils.get_file(
    fname="aclImdb.tar.gz", 
    origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
    extract=True,
)

84131840/84125825 [==============================] - 1s 0us/step


In [2]:
# "-".map(str, [1, 2, 3, 4, 5])

In [3]:
import glob
glob.glob("/root/.keras/datasets/aclImdb/train/pos/*")
import os
import pandas as pd


def read(path):
  with open(path, encoding = "utf8") as f :
    content = f.read()
  return content


def get_data(t):
    dn = os.path.dirname(dataset)
    pattern = os.path.join(dn, "aclImdb", t, "pos", "*.txt")
    pos = glob.glob(pattern)
    pattern = os.path.join(dn, "aclImdb", t, "neg", "*.txt")
    neg = glob.glob(pattern)
    sentiments = [1] * len(pos) + [0] * len(neg)
    contents = map(read, pos + neg)
    df = pd.DataFrame({
        "contents":contents,
        "sentiment":sentiments
    })
    return df

train_df = get_data("train")
test_df = get_data("test")
test_df

,contents,sentiment
0,"It kept me on the edge of my seat. True, the s...",1
1,Born in 1946 I was about eight years old when ...,1
2,In the 60's Cleveland television audiences cou...,1
3,I think it is a brilliant show with cool talki...,1
4,John Wayne's first starring role just blew me ...,1
...,...,...
24995,I am not a big fan of the Spielberg/Cruise ver...,0
24996,The director infuses this film with false dept...,0
24997,"As a flying and war movie buff, this ranks at ...",0
24998,I rented this movie under the impression that ...,0


embeddings:
https://keras.io/zh/layers/embeddings/

Tokenizer:



In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer

# 對文字的預處理
tok = Tokenizer(num_words = 3000)

# fit_on_tests: fit_transform 的 fit
tok.fit_on_texts(train_df["contents"])

# seg: a list with word trans to numbeer
x_train_seq = tok.texts_to_sequences(train_df["contents"])
x_test_seq = tok.texts_to_sequences(test_df["contents"])

In [5]:
# tok.word_index
tok.index_word[2]

'and'

In [6]:
pd.DataFrame(x_train_seq)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,1776,1777,1778,1779,1780,1781,1782,1783,1784,1785,1786,1787,1788,1789,1790,1791,1792,1793,1794,1795,1796,1797,1798,1799,1800,1801,1802,1803,1804,1805,1806,1807,1808,1809,1810,1811,1812,1813,1814,1815
0,1,17,6,84,2,10,37,1,62.0,10.0,2781.0,11.0,17.0,71.0,82.0,17.0,138.0,1.0,2753.0,1189.0,17.0,2.0,705.0,17.0,10.0,40.0,373.0,1.0,879.0,10.0,37.0,3.0,632.0,2.0,79.0,3.0,280.0,62.0,7.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11,6,4,4,84,399,209,105,11.0,215.0,3.0,2103.0,4.0,2537.0,1.0,2782.0,1.0,290.0,106.0,6.0,2186.0,16.0,1474.0,844.0,2.0,87.0,70.0,15.0,3.0,129.0,4.0,24.0,62.0,41.0,86.0,24.0,993.0,938.0,24.0,444.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,14,3,193,55,764,334,10,40,66.0,5.0,137.0,64.0,1.0,17.0,9.0,13.0,84.0,134.0,47.0,67.0,112.0,27.0,192.0,409.0,926.0,36.0,1.0,764.0,808.0,47.0,68.0,84.0,660.0,4.0,46.0,4.0,58.0,511.0,764.0,1838.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11,17,69,35,73,12,10,103,9.0,30.0,219.0,277.0,3.0,288.0,30.0,208.0,10.0,166.0,9.0,30.0,208.0,10.0,166.0,9.0,2.0,10.0,207.0,166.0,1.0,102.0,403.0,2.0,144.0,9.0,6.0,3.0,17.0,12.0,284.0,22.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11,1250,4,1976,1477,343,62,646,36.0,3.0,13.0,83.0,614.0,20.0,695.0,696.0,8.0,20.0,1.0,114.0,293.0,1305.0,2974.0,467.0,10.0,216.0,12.0,9.0,13.0,109.0,2444.0,171.0,20.0,2974.0,467.0,2.0,869.0,5.0,199.0,9.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,9,6,1295,12,3,226,14,1593,2.0,337.0,1753.0,97.0,94.0,98.0,265.0,57.0,1.0,389.0,28.0,1.0,411.0,8.0,11.0,17.0,163.0,303.0,37.0,3.0,309.0,1177.0,353.0,1511.0,295.0,1.0,75.0,2001.0,392.0,1643.0,1734.0,1298.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24996,3,19,37,477,1695,2,6,3,401.0,459.0,4.0,86.0,1.0,314.0,8.0,11.0,701.0,40.0,149.0,76.0,9.0,33.0,112.0,119.0,2.0,1992.0,5.0,1034.0,2.0,1034.0,80.0,989.0,7.0,7.0,11.0,19.0,2026.0,16.0,3.0,114.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24997,11,6,1,1124,4,592,12,10,25.0,107.0,8.0,3.0,193.0,55.0,1.0,233.0,55.0,10.0,1796.0,3.0,17.0,35.0,73.0,13.0,1861.0,146.0,501.0,300.0,2.0,47.0,6.0,424.0,54.0,210.0,5.0,11.0,17.0,546.0,5.0,64.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24998,10,1081,1884,12,143,1644,1,1106,2.0,920.0,42.0,3.0,2265.0,2322.0,2660.0,135.0,13.0,10.0,2765.0,119.0,10.0,566.0,58.0,416.0,39.0,2503.0,29.0,4.0,58.0,97.0,9.0,27.0,1155.0,7.0,7.0,54.0,10.0,20.0,704.0,134.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# truncating:pre 12345 => 45 , post => 12
from tensorflow.keras.preprocessing.sequence import pad_sequences
x_train_pad = pad_sequences(x_train_seq, 512)
x_test_pad = pad_sequences(x_test_seq, 512)
pd.DataFrame(x_test_pad)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,8,3,1161,12,1,17,6,21,1867,14,45,12,68,3,2178,15,49,19,280,54,39,1029,8,11,28,40,3,706,487,178,5,121,86,11,62,627,277,22,377,146
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,13,31,666,2041,1532,1,1437,606,37,3,247,8,58,702,9,13,37,316,32,776,877,9,90,3,524,133,12,73,430,276,54,28,77,166,11,730,18,9,1674,69
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,728,196,1,2,1,1393,1,106,13,1,29,55,115,265,1016,431,53,5,12,55,2,125,71,256,234,1558,6,305,7,7,92,171,48,78,10,121,143,61,3,504
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,4,792,23,134,22,200,64,86,9,492,8,1,1844,162,10,101,11,6,28,4,1,115,245,284,2,10,481,383,11,5,29,346,34,37,369,7,7,15,58,1911
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,97,1,254,248,6,32,1,995,1489,1847,23,2437,18,1,19,14,3,223,6,21,10,188,101,4,157,422,12,67,1556,1,191,8,3,278,4,578,476,20,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,10,822,12,230,2315,50,71,3,644,1755,8,1,624,485,307,35,5,10,479,10,25,5,137,16,1,30,219,9,405,1722,99,3,321,30,1,1710,1064,1214,1688,2335
24996,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,169,2752,2,26,57,8,108,47,23,197,131,104,99,1,290,1467,109,12,109,47,1098,364,2,149,20,1464,2,5,984,3,582,134,995,180,72,190,15,2484,8,110
24997,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2,12,1,84,81,33,23,266,5,27,162,119,11,91,217,14,45,33,801,5,898,8,14,108,2869,14,611,47,23,108,49,1544,105,2,11,6,21,28,4,95
24998,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,299,6,48,163,11,17,287,2739,7,7,1,315,941,1468,3,1234,646,80,1,174,2,1442,1919,4,1419,7,7,511,344,1,4,58,2037,177,5,11,52,558,4,55


In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers import Dense, Flatten, Dropout
# https://keras.io/zh/layers/embeddings/

layers = [
      # 1-3000(token) + 0(padding)
      Embedding(3001, 128, mask_zero=True, input_length=512),
      # Dense(128, activation="relu"),
      # Dropout(0.25),
      # over param. => over train
      GlobalAveragePooling1D(),
      Dense(2, activation="softmax")
]

model = Sequential(layers)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 512, 128)          384128    
_________________________________________________________________
global_average_pooling1d (Gl (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 2)                 258       
Total params: 384,386
Trainable params: 384,386
Non-trainable params: 0
_________________________________________________________________


In [9]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
model.compile(loss=SparseCategoricalCrossentropy(),
       optimizer=Adam(),
       metrics=["accuracy"])

In [10]:
y_train = train_df["sentiment"]
y_test = test_df["sentiment"]

In [11]:
# epoch: 整份資料要看幾遍
# 60000筆, 2epoch, 200batch -> 60000 * 2 / 200次調整
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
callbacks = [
    ModelCheckpoint("model.h5", save_best_only=True),
    EarlyStopping(patience=5, restore_best_weights=True)
]
model.fit(x_train_pad, 
     y_train,
     batch_size=100,
     epochs=50,
     validation_split=0.1,
     callbacks=callbacks,
     verbose=2)

Epoch 1/50
225/225 - 4s - loss: 0.5725 - accuracy: 0.7127 - val_loss: 0.4772 - val_accuracy: 0.7972
Epoch 2/50
225/225 - 4s - loss: 0.3619 - accuracy: 0.8620 - val_loss: 0.3867 - val_accuracy: 0.8356
Epoch 3/50
225/225 - 4s - loss: 0.2913 - accuracy: 0.8879 - val_loss: 0.3350 - val_accuracy: 0.8612
Epoch 4/50
225/225 - 4s - loss: 0.2595 - accuracy: 0.8992 - val_loss: 0.3476 - val_accuracy: 0.8544
Epoch 5/50
225/225 - 4s - loss: 0.2406 - accuracy: 0.9074 - val_loss: 0.3196 - val_accuracy: 0.8672
Epoch 6/50
225/225 - 4s - loss: 0.2279 - accuracy: 0.9137 - val_loss: 0.3390 - val_accuracy: 0.8596
Epoch 7/50
225/225 - 4s - loss: 0.2184 - accuracy: 0.9175 - val_loss: 0.3251 - val_accuracy: 0.8704
Epoch 8/50
225/225 - 4s - loss: 0.2118 - accuracy: 0.9199 - val_loss: 0.3877 - val_accuracy: 0.8392
Epoch 9/50
225/225 - 4s - loss: 0.2062 - accuracy: 0.9227 - val_loss: 0.3974 - val_accuracy: 0.8384
Epoch 10/50
225/225 - 4s - loss: 0.2021 - accuracy: 0.9233 - val_loss: 0.3452 - val_accuracy: 0.8636

In [12]:
model.evaluate(x_test_pad, y_test)

782/782 [==============================] - 3s 4ms/step - loss: 0.2869 - accuracy: 0.8830


[0.28689318895339966, 0.8829600214958191]

In [14]:
TOK = 3000
LEN = 512
EM = 128

In [15]:
layers = [
  Embedding(TOK+1, EM, mask_zero=True),
  GlobalAveragePooling1D()
]
w = model.layers[0].get_weights()
infer = Sequential(layers)
infer.layers[0].set_weights(w)
infer.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 128)         384128    
_________________________________________________________________
global_average_pooling1d_1 ( (None, 128)               0         
Total params: 384,128
Trainable params: 384,128
Non-trainable params: 0
_________________________________________________________________


In [16]:

# w
# infer.layers[0].get_weights()
# 25000, 512
# [512list, 512list]
target = "the"
# [1list]
pre = infer.predict([[tok.word_index[target]]])
pre[0]

array([-0.06068694, -0.01687202,  0.02700565, -0.00333076, -0.03431202,
       -0.01997601,  0.0151977 , -0.00860172, -0.03456532, -0.00127739,
       -0.04744357, -0.02280594, -0.03111822, -0.01839069,  0.05215816,
        0.04846331, -0.01904806,  0.04848907,  0.03078368,  0.0088834 ,
        0.04535828,  0.04563779, -0.05126249,  0.03175594,  0.02702455,
       -0.04778759,  0.06338232, -0.03956672,  0.00335033,  0.05556178,
       -0.06548061,  0.03358347,  0.03846428,  0.0042756 , -0.0197955 ,
        0.04338089,  0.0205578 ,  0.05643181,  0.02645518, -0.02831637,
       -0.01291166,  0.01663014,  0.05715456,  0.02498715, -0.03559098,
        0.0256614 , -0.00492434, -0.03036245, -0.01512167,  0.00419951,
        0.00640047,  0.01312962,  0.04808193, -0.00447035,  0.0128154 ,
        0.06373854,  0.0455902 , -0.0168565 ,  0.05186348, -0.00556332,
       -0.02385976, -0.01037923, -0.02948355, -0.02861106,  0.01022715,
       -0.06556471, -0.06553085,  0.00292578,  0.00496063,  0.01

In [17]:
f = open("vec.txt", "w", encoding="utf-8")
f.write("{} {}\n".format(TOK, EM))
for i in range(1, TOK+1):
    # [1list]
    pre = infer.predict([[i]])[0]
    v = " ".join(map(str, pre))
    f.write("{} {}\n".format(tok.index_word[i], v))
f.close()

In [ ]:
from gensim.models import KeyedVectors
w2v = KeyedVectors.load_word2vec_format("vec.txt")

In [ ]:
w2v.wv.vocab
w2v.most_similar("joke")                                                                                                                                     